In [13]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using SparseArrays
using ForwardDiff
using OSQP
using RobotDynamics
using RobotZoo: PlanarRocket
using RobotZoo
using StaticArrays
using Plots
using JLD2
using Test
using Statistics

const FD = ForwardDiff;

include("quadrotor.jl");

  Activating environment at `C:\Users\jonbs\Documents\JonathanStuff\CMU_Stuff\year2\OCRL\Quadrotor_MPC_with_Kalman_Filter\Project.toml`
Precompiling project...
  ✓ OSQP_jll
  ✓ ChainRulesCore
  ✓ DiffResults
  ✓ StructArrays
  ✓ FiniteDiff
  ✓ CoordinateTransformations
  ✓ Qt5Base_jll
  ✓ ArnoldiMethod
  ✓ ChangesOfVariables
  ✓ GR_jll
  ✓ Blink
  ✓ LogExpFunctions
  ✓ OSQP
  ✓ StatsBase
  ✓ GR
  ✓ Graphs
  ✓ GeometryBasics
  ✓ SpecialFunctions
  ✓ VertexSafeGraphs
  ✓ DiffRules
  ✓ DualNumbers
  ✓ Quaternions
  ✓ ForwardDiff
  ✓ Rotations
  ✓ SparseDiffTools
  ✓ RobotDynamics
  ✓ MeshCat
  ✓ RobotZoo
  ✓ Plots
  29 dependencies successfully precompiled in 44 seconds (164 already precompiled)
  26 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions


## Quadrotor Dynamics Model

The derivation of the 3d Quadrotor Dynamics model we use is available in [Lecture 15's Notes](https://github.com/Optimal-Control-16-745/lecture-notebooks/blob/main/Lecture%2015/Lecture%2015.pdf)

$$ 
x = \begin{bmatrix} {}^{N}r \\ q \\ {}^{B}V \\ {}^{B}\omega \end{bmatrix}, \quad
\dot{x} = 
\begin{bmatrix} 
    Q {}^{B}V \\
    \frac{1}{2} L(q) H {}^{B}\omega \\
    \frac{1}{m} {}^{B}F - {}^{B}\omega \times {}^{B}V \\
    J^{-1} ({}^{B}\tau - {}^{B}\omega \times J {}^{B}\omega)
\end{bmatrix}
$$

Where:
$$
{}^{B}F = Q^T
\begin{bmatrix}
0 \\ 0 \\ -mg
\end{bmatrix}
+ 
\begin{bmatrix}
    0 & 0 & 0 & 0 \\
    0 & 0 & 0 & 0 \\
    k_T & k_T & k_T & k_T
\end{bmatrix} u
$$

$$
{}^{B}\tau = 
\begin{bmatrix}
    0 & s k_T & 0 & -s k_T \\
    -s k_T & 0 & s k_T & 0 \\
    k_m & -k_m & k_m & -k_m
\end{bmatrix}
$$

Note that a rotation matrix $Q$ can be obtained from a quaternion $q$ via:
$$ 
Q = R^T(q) L(q)
$$

Such that:
$$
\begin{bmatrix} 0 \\ {}^{N}x \end{bmatrix} = Q \begin{bmatrix} 0 \\ {}^{B}x \end{bmatrix}
$$

For more information on the quaternion math, check out `src/quaternions.jl`

The model is defined like so:
![quad_dyn](../media/quadrotor_dynamics_image.png)
![quad_dyn_axes](../media/quadrotor_dynamics_axes_definition.png)

In [12]:
include("dynamics.jl")
# TODO: something to verify dynamics funcs work
nx = 6
nu = 2
dt = 0.025

x0 = zeros(nx)
uhover = fill(0.5*1.0 * 9.81, nu) # TODO: no magic numbers..?

A, B = dynamics_jacobians(x0, uhover, dt)

([1.0 0.0 … 0.0 2.5546875000000006e-5; 0.0 1.0 … 0.024999999999999998 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [-1.3305664062500004e-6 1.3305664062500004e-6; 0.00031250000000000006 0.00031250000000000006; … ; 0.024999999999999998 0.024999999999999998; -0.2083333333333334 0.2083333333333334])

In [ ]:
include("MPC.jl")

Q = I(nx)
R = I(nu)
Qf = I(nx)

Xref = nothing # TODO
Uref = nothing # TODO
tref = nothing

mpc = build_MPC_QP(Xref, Uref, tref, A, B, Q, R, Qf) 